In [2]:
import pandas as pd

pop = pd.read_csv('state-population.csv')   #各州人口数据
areas = pd.read_csv('state-areas.csv')     #各州面积数据
abbrevs = pd.read_csv('state-abbrevs.csv')   #各州名称缩写

print(pop.head());
print(areas.head());
print(abbrevs.head());

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [3]:
#想通过连接这些表，来计算每个州的人口密度，假设有3张表，在数据库中可以非常简单的计算出这些指标
#pop(state,ages,years,population)
#area(state,area)
#abb(state,abbreviation)
'''
    select 
        abb.state as state_name
        round(p.population*1.0/a.area,2) as pop_dense
    from
        pop p
    left join
        abb ab
    on
        p.state = abb.abbreviation
    left join
        area a
    on
        abb.state = a.state
    where
        p.ages = 'total';
'''
#首先对pop和abb进行外连接
merged = pd.merge(pop,abbrevs,how='outer',left_on = 'state/region',right_on='abbreviation')
merged = merged.drop('abbreviation',1)  #丢弃合并后的重复键列
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [4]:
#检查每个字段是否有确实值
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [5]:
#发现有些人就数据有空缺，需要重点查看
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [7]:
#也发现有一些州的全称是缺失的，检查去重列出
#如果是要切片出某一列，记得用loc
merged.loc[merged['state'].isnull(),'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [9]:
#可以通过后续方法补充全称
merged.loc[merged['state/region']=='PR','state'] = 'Puerto Rico'
merged.loc[merged['state/region']=='USA','state'] = 'United Stated'
#再次检查空缺情况
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [10]:
#这样state就没有缺失值了，可以将面积数据也合并进来
final = pd.merge(merged,areas,on='state',how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [11]:
#查看空缺情况
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [12]:
#查看哪些地区面积信息空缺
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United Stated'], dtype=object)

In [13]:
#可以插入所有面积的和作为全美面积，但由于我们只需要每个州的人口密度，所以可以抛弃有空值的数据
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [15]:
#完成数据准备后，可以用numexpr做快速运算,提取出2010年的数据
import numexpr

data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [16]:
#计算人口密度并排序
#首先对索引进行重置
data2010.set_index('state',inplace=True)
density = data2010['population']/data2010['area (sq. mi)']
data2010

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0
Colorado,CO,total,2010,5048196.0,104100.0
Connecticut,CT,total,2010,3579210.0,5544.0
Delaware,DE,total,2010,899711.0,1954.0
District of Columbia,DC,total,2010,605125.0,68.0


In [18]:
#降序排列，index还是州名
density.sort_values(ascending=False,inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [19]:
#查看密度最低的几个州
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64